In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive/courses/21_fall/ML/project")

In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import sys
import h5py
import numpy as np

In [2]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0, 2, 3, 1))

    return x_data, y_data


In [3]:
def data_preprocess(x_data):
    return x_data/255

In [11]:
clean_data_filename = './clean_validation_data.h5'
clean_testdata_filename = './clean_test_data.h5'

poisoned_data_filename_1 = './multi/eyebrows_poisoned_data.h5'
poisoned_data_filename_2 = './multi/lipstick_poisoned_data.h5'
poisoned_data_filename_3 = './multi/sunglasses_poisoned_data.h5'

model_filename = './multi/multi_trigger_multi_target_bd_net.h5'

cl_x_valid, cl_y_valid = data_loader(clean_data_filename)
cl_x_valid = data_preprocess(cl_x_valid)
cl_x_test, cl_y_test = data_loader(clean_testdata_filename)
cl_x_test = data_preprocess(cl_x_test)

eyebrows_x_test, eyebrows_y_test = data_loader(poisoned_data_filename_1)
eyebrows_x_test = data_preprocess(eyebrows_x_test)
lipstick_x_test, lipstick_y_test = data_loader(poisoned_data_filename_2)
lipstick_x_test = data_preprocess(lipstick_x_test)
sunglasses_x_test, sunglasses_y_test = data_loader(poisoned_data_filename_3)
sunglasses_x_test = data_preprocess(sunglasses_x_test)

net = keras.models.load_model(model_filename)
#net.load_weights('./multi/multi_trigger_multi_target_bd_weights.h5')

In [12]:
import pandas as pd

In [13]:
pool3 = net.get_layer('pool_3').output
pool3_net = keras.models.Model(net.input, pool3)
activation_all = pool3_net.predict(cl_x_valid)
print(activation_all.shape)

(11547, 5, 4, 60)


In [14]:
activation = np.mean(activation_all, axis=(0,1,2))
print(activation.shape)

(60,)


In [15]:
cl_label_or = np.argmax(net.predict(cl_x_valid), axis=1)
or_accuracy = np.mean(np.equal(cl_label_or, cl_y_valid))*100
print('Original Classification accuracy:', or_accuracy)

Original Classification accuracy: 96.26742876937733


Prune the b model according to the activation value of pooling layer. I set the channel threshold to 25, which means when the model will prune 25 channels.

In [16]:
#channel_threshold = 27
acc_threshold = 10

seq_sort = np.argsort(activation)

#conv3 = net.get_layer("conv_3")
#weight,bias = conv3.get_weights()

for i in range(60):
    channel = seq_sort[i]

    conv3 = net.get_layer("conv_3")
    weight,bias = conv3.get_weights()
    weight[:, :, :, channel] = 0. 
    bias[channel] = 0.
    conv3.set_weights((weight,bias))
    print("pruned channel:", channel)
    
    cl_label_v = np.argmax(net.predict(cl_x_valid), axis=1)
    valid_accuracy = np.mean(np.equal(cl_label_v, cl_y_valid))*100
    print('Clean Valid Classification accuracy:', valid_accuracy)

    if valid_accuracy < or_accuracy-acc_threshold:
      net.save("multi pruned model.h5")
      break
    
#net.save("sunglasses pruned model.h5")


pruned channel: 32
Clean Valid Classification accuracy: 96.26742876937733
pruned channel: 17
Clean Valid Classification accuracy: 96.26742876937733
pruned channel: 26
Clean Valid Classification accuracy: 96.26742876937733
pruned channel: 57
Clean Valid Classification accuracy: 96.26742876937733
pruned channel: 33
Clean Valid Classification accuracy: 96.27608902745301
pruned channel: 19
Clean Valid Classification accuracy: 96.30206980168009
pruned channel: 29
Clean Valid Classification accuracy: 96.25010825322595
pruned channel: 34
Clean Valid Classification accuracy: 96.1981467047718
pruned channel: 51
Clean Valid Classification accuracy: 96.16350567246904
pruned channel: 52
Clean Valid Classification accuracy: 95.9210184463497
pruned channel: 53
Clean Valid Classification accuracy: 94.92508876764528
pruned channel: 41
Clean Valid Classification accuracy: 94.84714644496405
pruned channel: 16
Clean Valid Classification accuracy: 94.75188360613146
pruned channel: 43
Clean Valid Classific

Fine tune the pruned model b prime.

In [ ]:
train_data_filename = './clean_validation_data.h5'
model_filename = './multi pruned model.h5'

x_train, y_train = data_loader(train_data_filename)
x_train = data_preprocess(x_train)

net = keras.models.load_model(model_filename)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
net.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

net.fit(x_train, y_train, epochs = 50)
net.save("multi fine-pruning model.h5")

In [ ]:
clean_data_filename = './clean_test_data.h5'

model_filename = './multi/multi_trigger_multi_target_bd_net.h5'
b_prime_model_filename = "./multi fine-pruning model.h5"

cl_x_test, cl_y_test = data_loader(clean_data_filename)
cl_x_test = data_preprocess(cl_x_test)


net = keras.models.load_model(model_filename)
b_prime_net = keras.models.load_model(b_prime_model_filename)

cl_b = np.argmax(net.predict(cl_x_test), axis=1)
cl_b_p = np.argmax(b_prime_net.predict(cl_x_test), axis=1)
cl_g = []
for x1, x2 in zip(cl_b, cl_b_p):
    if x1 == x2:
        cl_g.append(x1)
    else:
        cl_g.append(1283)

cl_g_acc = np.mean(np.equal(cl_g, cl_y_test))*100
print('Clean Classification accuracy for fine-pruning G net:', cl_g_acc)
    
    
    
eyebrows_bd_b = np.argmax(net.predict(eyebrows_x_test), axis=1)
eyebrows_bd_b_p = np.argmax(b_prime_net.predict(eyebrows_x_test), axis=1)
eyebrows_bd_g = []
for x1, x2 in zip(eyebrows_bd_b, eyebrows_bd_b_p):
    if x1 == x2:
        eyebrows_bd_g.append(x1)
    else:
        eyebrows_bd_g.append(1283)
        
eyebrows_asr = np.mean(np.equal(eyebrows_bd_g, eyebrows_y_test))*100
print('Attack Success Rate for fine-pruning G net on eyebrows poisoned data:', eyebrows_asr)


lipstick_bd_b = np.argmax(net.predict(lipstick_x_test), axis=1)
lipstick_bd_b_p = np.argmax(b_prime_net.predict(lipstick_x_test), axis=1)
lipstick_bd_g = []
for x1, x2 in zip(lipstick_bd_b, lipstick_bd_b_p):
    if x1 == x2:
        lipstick_bd_g.append(x1)
    else:
        lipstick_bd_g.append(1283)
        
lipstick_asr = np.mean(np.equal(lipstick_bd_g, lipstick_y_test))*100
print('Attack Success Rate for fine-pruning G net on lipstick poisoned data:', lipstick_asr)


sunglasses_bd_b = np.argmax(net.predict(sunglasses_x_test), axis=1)
sunglasses_bd_b_p = np.argmax(b_prime_net.predict(sunglasses_x_test), axis=1)
sunglasses_bd_g = []
for x1, x2 in zip(sunglasses_bd_b, sunglasses_bd_b_p):
    if x1 == x2:
        sunglasses_bd_g.append(x1)
    else:
        sunglasses_bd_g.append(1283)
        
sunglasses_asr = np.mean(np.equal(sunglasses_bd_g, sunglasses_y_test))*100
print('Attack Success Rate for fine-pruning G net on sunglasses poisoned data:', sunglasses_asr)